In [71]:
# I love Google Colab

In [72]:
#get the data
!rm -rf age_prediction/
%ls
!rm -rf data
%ls
!unzip data.zip

Streaming output truncated to the last 5000 lines.
  inflating: data/faces/39411334@N02/coarse_tilt_aligned_face.413.11043392003_2ebc348d15_o.jpg  
  inflating: data/faces/39411334@N02/coarse_tilt_aligned_face.413.11043451425_37d539658c_o.jpg  
  inflating: data/faces/39411334@N02/coarse_tilt_aligned_face.413.11043452385_cff2f2e58d_o.jpg  
  inflating: data/faces/39411334@N02/coarse_tilt_aligned_face.413.11043551486_eff2f85c03_o.jpg  
  inflating: data/faces/39411334@N02/coarse_tilt_aligned_face.413.11043782576_4fed4bc6c5_o.jpg  
  inflating: data/faces/39411334@N02/coarse_tilt_aligned_face.413.11043711335_1e545d5d40_o.jpg  
  inflating: data/faces/39411334@N02/coarse_tilt_aligned_face.413.11043727513_790eee3944_o.jpg  
  inflating: data/faces/39411334@N02/coarse_tilt_aligned_face.413.11043781116_63f39ba76b_o.jpg  
  inflating: data/faces/39411334@N02/coarse_tilt_aligned_face.414.11043790344_a6df9011e1_o.jpg  
  inflating: data/faces/39411334@N02/coarse_tilt_aligned_face.413.1104381907

In [73]:

import numpy as np
import cv2
IMG_SIZE=100

def create_training_data(x):
	l=[]
	text_file_name="./data/fold_"+str(x)+"_data.txt"
	text_file = open(text_file_name, "r")
	lines = text_file.readlines()
	for line in lines:
		ls=line.split("\t")
		file_name="./data/faces/"+ls[0]+'/coarse_tilt_aligned_face.'+ls[2]+"."+ls[1]
		temp=ls[3].replace(" ","")[1:-1].split(",")
		valid_age=-1
		valid1=["0","4","8","15","25","38","48","60"]
		for i in range(len(valid1)):
			if valid1[i]==temp[0]:
				valid_age=i

		valid_gender=-1
		valid2=["m","f","u"]
		for i in range(len(valid2)):
			if valid2[i]==ls[4]:
				valid_gender=i

		# Using 0 to read image in grayscale mode
		# img = cv2.imread(file_name,0)
		img = cv2.imread(file_name)
		if valid_gender!=-1 and valid_age!=-1 and (img is not None) and img.shape[0]==227 and img.shape[1]==227:
			# img=cv2.resize(img,(IMG_SIZE,IMG_SIZE))
	 		l.append([np.array(img),valid_gender,valid_age])

	text_file.close()
	return l

In [74]:
import tensorflow as tf
tf.test.gpu_device_name()
## how much gpu is avalable to you by seeing last line
# # memory footprint support libraries/code
# !ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
# !pip install gputil
# !pip install psutil
# !pip install humanize
# import psutil
# import humanize
# import os
# import GPUtil as GPU
# GPUs = GPU.getGPUs()
# # XXX: only one GPU on Colab and isn’t guaranteed
# gpu = GPUs[0]
# def printm():
#  process = psutil.Process(os.getpid())
#  print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
#  print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
# printm() 
print("========================================")
valid_X = []
valid_Y_gender = []
valid_Y_age = []
train_X = []
train_Y_gender = []
train_Y_age = []
test_X=[]
test_Y_gender=[]
test_Y_age=[]

temp=[]
for fold_id in [0,1,2,3]:
	print("processing ... ",fold_id)
	temp.extend(create_training_data(fold_id))


temp=np.array(temp)
np.random.shuffle(temp)

for i in temp[:int(len(temp)*(0.9))]:
	train_X.append(i[0])
	train_Y_gender.append(i[1])
	train_Y_age.append(i[2])


for i in temp[int(len(temp)*(0.9)):]:
	valid_X.append(i[0])
	valid_Y_gender.append(i[1])
	valid_Y_age.append(i[2])


temp=create_training_data(4)
temp=np.array(temp)
np.random.shuffle(temp)

for i in temp:
	test_X.append(i[0])
	test_Y_gender.append(i[1])
	test_Y_age.append(i[2])

train_X=np.array(train_X)
train_Y_age=np.array(train_Y_age)
train_Y_gender=np.array(train_Y_gender)

valid_X=np.array(valid_X)
valid_Y_age=np.array(valid_Y_age)
valid_Y_gender=np.array(valid_Y_gender) 

test_X=np.array(test_X)
test_Y_age=np.array(test_Y_age)
test_Y_gender=np.array(test_Y_gender)



processing ...  0
processing ...  1
processing ...  2
processing ...  3


In [75]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
# import matplotlib.pyplot as plt
# import os
# os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

model_age = models.Sequential()
#CONV1
# model_age.add(layers.Conv2D(32, (7, 7), activation='relu', input_shape=(100, 100, 1)))
model_age.add(layers.Conv2D(96, (7, 7),strides=(4,4), input_shape=(227, 227, 3)))
model_age.add(layers.Activation('relu'))
model_age.add(layers.BatchNormalization())
model_age.add(layers.MaxPooling2D(pool_size=(2, 2)))
model_age.add(layers.Dropout(0.5))
#CONV2
model_age.add(layers.Conv2D(256, (5, 5)))
model_age.add(layers.Activation('relu'))
model_age.add(layers.BatchNormalization())
model_age.add(layers.MaxPooling2D(pool_size=(2, 2)))
model_age.add(layers.Dropout(0.5))
#CONV3
model_age.add(layers.Conv2D(384, (3, 3)))
model_age.add(layers.Activation('relu'))
model_age.add(layers.BatchNormalization())
model_age.add(layers.MaxPooling2D(pool_size=(2, 2)))
model_age.add(layers.Dropout(0.5))
#CONV4
model_age.add(layers.Conv2D(256, (3, 3)))
model_age.add(layers.Activation('relu'))
model_age.add(layers.BatchNormalization())
model_age.add(layers.MaxPooling2D(pool_size=(3, 3)))
model_age.add(layers.Dropout(0.5))
#flat
model_age.add(layers.Flatten())
# FC1
model_age.add(layers.Dense(512))
model_age.add(layers.Activation('relu'))
model_age.add(layers.BatchNormalization())
model_age.add(layers.Dropout(0.5))
#FC2
model_age.add(layers.Dense(8))

# model_age.summary()

model_age.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])


history = model_age.fit(train_X, train_Y_age, epochs=200,
                    validation_data=(valid_X, valid_Y_age))

model_age.save('age_prediction')
model_age.save('age_prediction.h5')

Epoch 1/200
325/325 [==============================] - 17s 51ms/step - loss: 2.5141 - accuracy: 0.2222 - val_loss: 1.8379 - val_accuracy: 0.2794
Epoch 2/200
325/325 [==============================] - 16s 50ms/step - loss: 1.9523 - accuracy: 0.3132 - val_loss: 1.5894 - val_accuracy: 0.4239
Epoch 3/200
325/325 [==============================] - 16s 50ms/step - loss: 1.6909 - accuracy: 0.3729 - val_loss: 1.5264 - val_accuracy: 0.4014
Epoch 4/200
325/325 [==============================] - 16s 50ms/step - loss: 1.5478 - accuracy: 0.4198 - val_loss: 1.4313 - val_accuracy: 0.4732
Epoch 5/200
325/325 [==============================] - 16s 50ms/step - loss: 1.4832 - accuracy: 0.4413 - val_loss: 1.4671 - val_accuracy: 0.4429
Epoch 6/200
325/325 [==============================] - 16s 50ms/step - loss: 1.4290 - accuracy: 0.4555 - val_loss: 1.4283 - val_accuracy: 0.4611
Epoch 7/200
325/325 [==============================] - 16s 50ms/step - loss: 1.3834 - accuracy: 0.4688 - val_loss: 1.3878 - val_ac

In [76]:
# 325/325 [==============================] - 17s 52ms/step - loss: 0.1854 - accuracy: 0.9318 - val_loss: 0.6293 - val_accuracy: 0.8304
# INFO:tensorflow:Assets written to: age_prediction/assets
test_loss, test_acc = model_age.evaluate(test_X,  test_Y_age, verbose=2)
print("\n\nAccuracy(Age Prediction) on Test images ==> ",test_acc*100,"% \n")
import numpy as np
import cv2

def get_exact1off_acc1(test_X,test_Y):
  result = model_age.predict_classes(test_X)
  correctly_classified=0
  for i in range(len(result)):
    if(abs(result[i]-test_Y[i])==1 or abs(result[i]-test_Y[i])==0):
      correctly_classified+=1

  return ((correctly_classified/len(result))*100)

print("\n\nExact 1-off Accuracy(Age Prediction) on Test images ==> ",get_exact1off_acc1(test_X,test_Y_age),"% \n")

91/91 - 1s - loss: 2.0581 - accuracy: 0.5066


Accuracy(Age Prediction) on Test images ==>  50.65835118293762 % 



Exact 1-off Accuracy(Age Prediction) on Test images ==>  81.70478170478171 % 



In [77]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
# import matplotlib.pyplot as plt
# import os
# os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

model_gender = models.Sequential()
#CONV1
# model_gender.add(layers.Conv2D(32, (7, 7), activation='relu', input_shape=(100, 100, 1)))
model_gender.add(layers.Conv2D(96, (7, 7),strides=(4,4), input_shape=(227, 227, 3)))
model_gender.add(layers.Activation('relu'))
model_gender.add(layers.BatchNormalization())
model_gender.add(layers.MaxPooling2D(pool_size=(2, 2)))
model_gender.add(layers.Dropout(0.5))
#CONV2
model_gender.add(layers.Conv2D(256, (5, 5)))
model_gender.add(layers.Activation('relu'))
model_gender.add(layers.BatchNormalization())
model_gender.add(layers.MaxPooling2D(pool_size=(2, 2)))
model_gender.add(layers.Dropout(0.5))
#CONV3
model_gender.add(layers.Conv2D(384, (3, 3)))
model_gender.add(layers.Activation('relu'))
model_gender.add(layers.BatchNormalization())
model_gender.add(layers.MaxPooling2D(pool_size=(2, 2)))
model_gender.add(layers.Dropout(0.5))
#CONV4
model_gender.add(layers.Conv2D(256, (3, 3)))
model_gender.add(layers.Activation('relu'))
model_gender.add(layers.BatchNormalization())
model_gender.add(layers.MaxPooling2D(pool_size=(3, 3)))
model_gender.add(layers.Dropout(0.5))
#flat
model_gender.add(layers.Flatten())
# FC1
model_gender.add(layers.Dense(512))
model_gender.add(layers.Activation('relu'))
model_gender.add(layers.BatchNormalization())
model_gender.add(layers.Dropout(0.5))
#FC2
model_gender.add(layers.Dense(3))

# model_gender.summary()

model_gender.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])


history = model_gender.fit(train_X, train_Y_gender, epochs=200,
                    validation_data=(valid_X, valid_Y_gender))

model_gender.save('gender_prediction')
model_gender.save('gender_prediction.h5')

Epoch 1/200
325/325 [==============================] - 17s 52ms/step - loss: 1.1759 - accuracy: 0.5087 - val_loss: 0.8053 - val_accuracy: 0.5744
Epoch 2/200
325/325 [==============================] - 17s 51ms/step - loss: 0.8528 - accuracy: 0.5883 - val_loss: 0.7131 - val_accuracy: 0.6566
Epoch 3/200
325/325 [==============================] - 17s 51ms/step - loss: 0.7611 - accuracy: 0.6259 - val_loss: 0.6645 - val_accuracy: 0.6661
Epoch 4/200
325/325 [==============================] - 17s 51ms/step - loss: 0.7066 - accuracy: 0.6585 - val_loss: 0.6534 - val_accuracy: 0.6696
Epoch 5/200
325/325 [==============================] - 17s 51ms/step - loss: 0.6548 - accuracy: 0.6875 - val_loss: 0.6623 - val_accuracy: 0.6505
Epoch 6/200
325/325 [==============================] - 17s 51ms/step - loss: 0.6274 - accuracy: 0.7105 - val_loss: 0.6034 - val_accuracy: 0.7042
Epoch 7/200
325/325 [==============================] - 17s 51ms/step - loss: 0.6008 - accuracy: 0.7240 - val_loss: 0.5314 - val_ac

In [78]:

test_loss, test_acc = model_gender.evaluate(test_X,  test_Y_gender, verbose=2)
print("\n\nAccuracy(Gender Prediction) on Test images ==> ",test_acc*100,"% \n")

91/91 - 1s - loss: 0.7483 - accuracy: 0.8157


Accuracy(Gender Prediction) on Test images ==>  81.56617879867554 % 

